# yo-yo-maskr
## Part 1: Regex based anonymization

In [ ]:
import dill
import regex as re
from src.utils.ano_regex import create_names_regex
from src.utils.trie import Trie

NAMES_FROM_PICKLE = False
USE_FULL = False

if NAMES_FROM_PICKLE:
    with open('./data/first_names_regex.pkl', 'rb') as f:
        first_names = dill.load(f)
    with open('./data/last_names_regex.pkl', 'rb') as f:
        last_names = dill.load(f)
    with open('./data/first_names_trie_regex.pkl', 'rb') as f:
        first_names_trie = dill.load(f)
    with open('./data/last_names_trie_regex.pkl', 'rb') as f:
        last_names_trie = dill.load(f)
else:
    with open(f'./data/first_names{"_full" if USE_FULL else ""}.txt') as f:
        first_names = [l.strip() for l in f.read().split('\n') if len(l.strip()) > 0]
    first_names_regex = create_names_regex(first_names)
    with open('./data/first_names_regex.pkl', 'wb') as f:
            dill.dump(first_names_regex, f)
    first_trie = Trie()
    for name in sorted([f for f in first_names if len(f) > 0], key=len, reverse=True):
        first_trie.add(name)
    first_trie_regex = re.compile(r'\b' + first_trie.pattern() + r'\b')
    with open('./data/first_names_trie_regex.pkl', 'wb') as f:
        dill.dump(first_trie_regex, f)

    with open(f'./data/last_names{"_full" if USE_FULL else ""}.txt') as f:
        last_names = [l.strip() for l in f.read().split('\n') if len(l.strip()) > 0]
    last_names_regex = create_names_regex(last_names)
    with open('./data/last_names_regex.pkl', 'wb') as f:
        dill.dump(last_names_regex, f)
    last_trie = Trie()
    for name in sorted([l for l in last_names if len(l) > 0], key=len, reverse=True):
        last_trie.add(name)
    last_trie_regex = re.compile(r'\b' + last_trie.pattern() + r'\b')
    with open('./data/last_names_trie_regex.pkl', 'wb') as f:
        dill.dump(last_trie_regex, f)
    

In [20]:
import regex as re
from tqdm.auto import tqdm
from src.utils.ano_regex import anonymize_entities

with open('./sample_texts.txt') as f:
    sample_texts = [f.read()]
    sample_texts = f.read().split('\n')

result = [anonymize_entities(text, by_names='NAME', first_names=first_trie_regex, last_names=last_trie_regex)
          for text in tqdm(sample_texts[:])]



  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
len(result[0].get('replace_dict'))

146

In [ ]:
from flashtext2 import KeywordProcessor
from tqdm.auto import tqdm

ln = KeywordProcessor(case_sensitive=True)
for name in last_names:
    ln.add_keyword(name)

fn = KeywordProcessor(case_sensitive=True)
for name in first_names:
    fn.add_keyword(name)

result = [{'text': text, 'first_names': fn.extract_keywords_with_span(text),
           'last_names': ln.extract_keywords_with_span(text)} for text in tqdm(sample_texts)]

In [ ]:
from retrie.trie import Trie

retrie = Trie()
for name in sorted([l for l in last_names if len(l) > 0], key=len, reverse=True):
    retrie.add(name)

retrie.pattern()